In [17]:
import os
import re

import random
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

from keras.models import Model 
from keras.layers.recurrent import LSTM
from keras.layers import Input, Dense, Dropout
from keras.callbacks import ModelCheckpoint,Callback

In [18]:
data = open(os.path.join("data","Quotables","author-quote.txt"), 'ra').read()
data = data.decode("utf-8")

data = re.split(r"[\t\n]", data)[1::2]
for i,d in enumerate(data):
    print d
    if i>5:
        break

data = " ".join(data)     
labels = list(set(data))

max_len = 20
nr_classes = len(labels)
MODEL_FILEPATH = os.path.join("models","quote_rnn.h5")
DICT_FILEPATH = os.path.join("models","quote_helper_dicts.pkl")

text2label = {t:i for i, t in enumerate(labels)}
label2text = {i:t for i, t in enumerate(labels)}
joblib.dump((text2label,label2text),DICT_FILEPATH)

data = data[:1000]

If you live to be a hundred, I want to live to be a hundred minus one day so I never have to live without you.
Promise me you'll always remember: You're braver than you believe, and stronger than you seem, and smarter than you think.
Did you ever stop to think, and forget to start again?
Organizing is what you do before you do something, so that when you do it, it is not all mixed up.
Weeds are flowers too, once you get to know them.
You can't stay in your corner of the forest waiting for others to come to you. You have to go to them sometimes.
The third-rate mind is only happy when it is thinking with the majority. The second-rate mind is only happy when it is thinking with the minority. The first-rate mind is only happy when it is thinking.


In [19]:
step = 3
inputs,outputs = [], []
for i in range(0, len(data) - max_len, step):
    inputs.append(data[i:i+max_len])
    outputs.append(data[i+max_len])

X = np.zeros((len(inputs), max_len, len(label2text)))
y = np.zeros((len(inputs), len(label2text)))

for i, example in enumerate(inputs):
    for t, label in enumerate(example):
        X[i, t, text2label[label]] = 1
    y[i, text2label[outputs[i]]] = 1
    
X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size = 0.8, 
                                                   random_state = 1234) 

In [20]:
text_input = Input(shape=(max_len, nr_classes))
x = LSTM(64, return_sequences=True)(text_input)
x = Dropout(0.2)(x)
x = LSTM(64, return_sequences=False)(x)
x = Dropout(0.2)(x)
text_output = Dense(nr_classes,activation='softmax')(x)

char_rnn_model = Model(text_input,text_output)
char_rnn_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
checkpoint = ModelCheckpoint(filepath=MODEL_FILEPATH,monitor='val_loss', verbose=0, 
                             save_best_only=True, save_weights_only=False, mode='auto')

char_rnn_model.fit(X_train, y_train,
                   validation_data = [X_valid,y_valid],
                   batch_size=128, 
                   nb_epoch=10,
                   callbacks = [checkpoint])

Train on 261 samples, validate on 66 samples
Epoch 1/10
261/261 [==============================] - 3s - loss: 4.4505 - val_loss: 4.4000
Epoch 2/10
261/261 [==============================] - 1s - loss: 4.3954 - val_loss: 4.3388
Epoch 3/10
261/261 [==============================] - 1s - loss: 4.3257 - val_loss: 4.2451
Epoch 4/10
261/261 [==============================] - 1s - loss: 4.2158 - val_loss: 4.1009
Epoch 5/10
261/261 [==============================] - 1s - loss: 4.0847 - val_loss: 3.9177
Epoch 6/10
261/261 [==============================] - 1s - loss: 3.9099 - val_loss: 3.7518
Epoch 7/10
261/261 [==============================] - 1s - loss: 3.7487 - val_loss: 3.6139
Epoch 8/10
261/261 [==============================] - 1s - loss: 3.6210 - val_loss: 3.4937
Epoch 9/10
261/261 [==============================] - 1s - loss: 3.4859 - val_loss: 3.3915
Epoch 10/10
261/261 [==============================] - 1s - loss: 3.3687 - val_loss: 3.3144
